# LinkedIn Profile Analysis

In [11]:
import numpy as np
import pandas as pd

import names


In [12]:
data = pd.read_csv("./profiles.csv")
data = data.drop_duplicates(subset='user')
data['user'] = data['user'].transform(lambda x: names.get_full_name())
data.head(20)


,user,posts,titles
0,Jose Mathews,Skills: Microsoft Excel · Microsoft Word · Mic...,IT Business Partner w Seargin | Coordinating G...
2,Aileen Wynn,Employer Branding;CommunicationsHuman Resource...,Chief Executive Officer
4,Joshua Clowes,We are still ; hashtag; #hiring; ...,Talent Accquistion
6,Vanessa Alvares,Érdekel a tőzsde világa; de nincs időd követni...,Senior Financial Markets Specialist at KBC Sec...
8,Johnny Stroman,Skills: Natural Language Processing (NLP) · Al...,Data Scientist at BlackRock
10,Ashleigh Rosenblatt,Congratulations Sukumar ..;Buying & Merchandis...,Buying | Merchandising | Category Management |...
12,Carey Jacobs,Prime Brokerage / Stock Loan teamFixed Income ...,Vice President at BlackRock
14,Max Warren,Congratulations!;Gratulálok; Eszter!;Székely C...,People Manager / HR & Recruitment Manager / Bu...
16,Joseph Chestnut,Challenge the challenge;Sharks intervention;A ...,Product Design | Product Analysis | Product Ma...
18,Carmen Urbine,I’m disheartened to hear about the recent layo...,Agile Coach at Ozow
